<a href="https://colab.research.google.com/github/olaidejoseph/Hamoye-2/blob/master/Acc_91_8_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Kaggle"

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
! ls

drive				     sample_data
Kaggle-planet-test-tif.torrent	     sample_submission_v2.csv
Kaggle-planet-test-tif.torrent.zip   sample_submission_v2.csv.zip
Kaggle-planet-train-tif.torrent      test_v2_file_mapping.csv
Kaggle-planet-train-tif.torrent.zip  train_v2.csv
planets-dataset.zip		     train_v2.csv.zip


In [21]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np
import matplotlib.pyplot as plt
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import cv2
%matplotlib inline
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD, RMSprop, Adam 
from keras.utils import np_utils
from keras.layers.core import Dropout

# If you would like to make further imports from tensorflow, add them here

from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import  Sequential, Model

In [186]:
#!kaggle competitions download -c planet-understanding-the-amazon-from-space

In [187]:
#!kaggle datasets download -d nikitarom/planets-dataset

In [185]:
#!unzip \*.zip  && rm *.zip

In [68]:
df_train = pd.read_csv('train_v2.csv')

In [69]:
df_test = pd.read_csv('sample_submission_v2.csv')

In [89]:
train_data = df_train[:40479-4500]
val_data = df_train[40479-4500:]

In [90]:
from tqdm import tqdm


y_train = []


flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    y_train.append(targets)

100%|██████████| 40479/40479 [00:00<00:00, 286968.58it/s]


In [91]:
y_train = np.array(y_train, np.uint8)

In [92]:
y_train1 = y_train[:40479-4500]
y_val = y_train[40479-4500:]
y_train = y_train1

In [142]:
# Create a function that returns an infinitely looping generator for the train set


def get_generator_cyclic(features, labels, batch_size=1):
  while True:
    for n in range(len(features)//batch_size):
      sd = []
      for e in train_data['image_name'][n*batch_size:(n+1)*batch_size]:
        img = cv2.imread('/content/planet/planet/train-jpg/{}.jpg'.format(e))
        sd.append(cv2.resize(img, (299,299)))
      cd = np.array(sd, np.float32)/255.
      yield (cd, labels[n*batch_size: (n+1)*batch_size])
      sd = []
  


In [143]:
zx = get_generator_cyclic(train_data, y_train, batch_size=128)

In [144]:
# Create a function that returns an infinitely looping generator for the validation set


def get_valgenerator_cyclic(features, labels, batch_size=1):
  while True:
    for n in range(len(features)//batch_size):
      sd = []
      for e in val_data['image_name'][n*batch_size:(n+1)*batch_size]:
        img = cv2.imread('/content/planet/planet/train-jpg/{}.jpg'.format(e))
        sd.append(cv2.resize(img, (299,299)))
      cd = np.array(sd, np.float32)/255.
      yield (cd, labels[n*batch_size: (n+1)*batch_size])
      sd = []
  

In [145]:
valzx = get_valgenerator_cyclic(val_data, y_val, batch_size=128)

In [146]:
# Create a function that returns an infinitely looping generator for the test set 

def get_test_generator_cyclic(features, batch_size=1):
  while True:
    for n in range(len(df_test['image_name'].iloc[:40669])//batch_size):
      sd = []
      for e in df_test['image_name'].iloc[:40669][n*batch_size:(n+1)*batch_size]:
        img = cv2.imread('/content/planet/planet/test-jpg/{}.jpg'.format(e))
        sd.append(cv2.resize(img, (299,299)))
      cd = np.array(sd, np.float32)/255.  
      yield (cd)
      sd = []
    for ni in range(len(df_test['image_name'].iloc[:40669])//batch_size):
      sa = []
      for w in df_test['image_name'].iloc[40669:][ni*batch_size:(ni+1)*batch_size]:
        img2 = cv2.imread('/content/test-jpg-additional/test-jpg-additional/{}.jpg'.format(w))
        sa.append(cv2.resize(img2, (299,299)))      
      cds = np.array(sa, np.float32)/255.
      yield (cds)
      sa = []

In [160]:
testzz1 = get_test_generator_cyclic(df_test, batch_size=128)

In [148]:
import os

base_model = tf.keras.applications.InceptionResNetV2(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [149]:
base_model.trainable = False

In [150]:
inputs = Input(shape=(299, 299, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
# A Dense classifier with (multilabel classification)
outputs = Dense(17, activation='sigmoid')(x)
model = Model(inputs, outputs)

In [151]:
from keras import backend


def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

In [152]:
model.compile(optimizer=Adam(0.0001),
              loss='binary_crossentropy',
              metrics=[fbeta])

In [153]:
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 299, 299, 3)]     0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 8, 8, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1536)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              1573888   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 17)                17425     
Total params: 55,928,049
Trainable params: 1,591,313
Non-trainable params: 54,336,736
_________________________________

In [154]:
train_steps_per_epoch = len(y_train) // 128
print(train_steps_per_epoch)
val_steps_per_epoch = len(y_val) // 128
print(val_steps_per_epoch)

281
35


In [156]:
# Creating Callbacks 
checkpoint_best_path = '/content/drive/My Drive/Kaggle'

checkpoint_best = ModelCheckpoint(filepath=checkpoint_best_path , save_weights_only=True,
                                  save_best_only = True, verbose=1,save_freq='epoch', 
                                  monitor= 'val_fbeta', mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(patience=7,monitor='val_fbeta', mode='max')
lrp = tf.keraslrp = tf.keras.callbacks.ReduceLROnPlateau(factor=0.25 , min_lr=0.0001)



# fitting
history = model.fit(
            zx,
            steps_per_epoch = train_steps_per_epoch, validation_data=valzx, 
            validation_steps=val_steps_per_epoch,
            epochs = 40,
            verbose = 2, callbacks=[early_stopping, lrp, checkpoint_best])

Epoch 1/40

Epoch 00001: val_fbeta improved from -inf to 0.82857, saving model to /content/drive/My Drive/Kaggle
281/281 - 408s - loss: 0.1699 - fbeta: 0.7790 - val_loss: 0.1293 - val_fbeta: 0.8286
Epoch 2/40

Epoch 00002: val_fbeta improved from 0.82857 to 0.84372, saving model to /content/drive/My Drive/Kaggle
281/281 - 404s - loss: 0.1293 - fbeta: 0.8308 - val_loss: 0.1204 - val_fbeta: 0.8437
Epoch 3/40

Epoch 00003: val_fbeta improved from 0.84372 to 0.85428, saving model to /content/drive/My Drive/Kaggle
281/281 - 405s - loss: 0.1217 - fbeta: 0.8420 - val_loss: 0.1160 - val_fbeta: 0.8543
Epoch 4/40

Epoch 00004: val_fbeta improved from 0.85428 to 0.85909, saving model to /content/drive/My Drive/Kaggle
281/281 - 404s - loss: 0.1173 - fbeta: 0.8503 - val_loss: 0.1131 - val_fbeta: 0.8591
Epoch 5/40

Epoch 00005: val_fbeta improved from 0.85909 to 0.86250, saving model to /content/drive/My Drive/Kaggle
281/281 - 404s - loss: 0.1143 - fbeta: 0.8542 - val_loss: 0.1110 - val_fbeta: 0.862

In [161]:
zx = get_generator_cyclic(train_data, y_train, batch_size=64)
valzx = get_valgenerator_cyclic(val_data, y_val, batch_size=64)

In [162]:
train_steps_per_epoch = len(y_train) // 64
print(train_steps_per_epoch)
val_steps_per_epoch = len(y_val) // 64
print(val_steps_per_epoch)

562
70


In [163]:
# fine-tuning

base_model.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[fbeta])


early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
lrp = tf.keraslrp = tf.keras.callbacks.ReduceLROnPlateau(factor=0.2)

history = model.fit(
            zx,
            steps_per_epoch = train_steps_per_epoch, validation_data=valzx, 
            validation_steps=val_steps_per_epoch,
            epochs = 2,
            verbose = 2, callbacks=[early_stopping, lrp])

Epoch 1/2
562/562 - 1287s - loss: 0.0853 - fbeta: 0.8976 - val_loss: 0.0932 - val_fbeta: 0.8917
Epoch 2/2
562/562 - 1286s - loss: 0.0746 - fbeta: 0.9109 - val_loss: 0.0935 - val_fbeta: 0.8946


In [164]:
p_tes1 = model.predict(testzz1, verbose=1, steps=477)

477/477 [==============================] - 608s 1s/step


In [165]:
p_tes1.shape

(61056, 17)

In [166]:
# Creating an infinite looping generator for my test set offset

def get_test3_generator_cyclic(labels, batch_size=1):
  while True:
    for ni in range(len(df_test['image_name'].iloc[40576:40669])//batch_size):
      sa = []
      for w in df_test['image_name'].iloc[40576:40669][ni*batch_size:(ni+1)*batch_size]:
        img2 = cv2.imread('/content/planet/planet/test-jpg/{}.jpg'.format(w))
        sa.append(cv2.resize(img2, (299,299)))      
      cds = np.array(sa, np.float32)/255.
      yield (cds)
      sa = []


def get_test4_generator_cyclic(labels, batch_size=1):
  while True:
    for ni in range(len(df_test['image_name'].iloc[61149:61191])//batch_size):
      sa = []
      for w in df_test['image_name'].iloc[61149:61191][ni*batch_size:(ni+1)*batch_size]:
        img2 = cv2.imread('/content/test-jpg-additional/test-jpg-additional/{}.jpg'.format(w))
        sa.append(cv2.resize(img2, (299,299)))      
      cds = np.array(sa, np.float32)/255.
      yield (cds)
      sa = []

In [167]:
testzz3 = get_test3_generator_cyclic(df_test, batch_size=31)
testzz4 = get_test4_generator_cyclic(df_test, batch_size=7)

In [168]:
p_tes3 = model.predict(testzz3, verbose=1, steps=3)

3/3 [==============================] - 1s 215ms/step


In [169]:
p_tes3.shape

(93, 17)

In [170]:
p_tes4 = model.predict(testzz4, verbose=1, steps=6)

6/6 [==============================] - 0s 72ms/step


In [171]:
p_tes4.shape

(42, 17)

In [172]:
y_test = []
y_test.append(p_tes1)

result = np.array(y_test[0])
result = pd.DataFrame(result, columns=labels)

In [173]:
y_test3 = []
y_test3.append(p_tes3)

result3 = np.array(y_test3[0])
result3 = pd.DataFrame(result3, columns=labels)

In [174]:
y_test4 = []
y_test4.append(p_tes4)

result4 = np.array(y_test4[0])
result4 = pd.DataFrame(result4, columns=labels)

In [175]:
preds4 = []

for i in tqdm(range(result4.shape[0]), miniters=1000):
    a = result4.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds4.append(' '.join(list(a.index)))

100%|██████████| 42/42 [00:00<00:00, 302.24it/s]


In [176]:
preds3 = []

for i in tqdm(range(result3.shape[0]), miniters=1000):
    a = result3.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds3.append(' '.join(list(a.index)))

100%|██████████| 93/93 [00:00<00:00, 363.85it/s]


In [177]:
preds = []

for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.iloc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61056/61056 [02:39<00:00, 383.22it/s]


In [178]:
fist_df = pd.DataFrame(df_test['image_name'][:40576])
fist_df['tags'] = preds[:40576]

In [179]:
fist_df3 = pd.DataFrame(df_test['image_name'][40576:40669]).reset_index(drop=True)
fist_df3['tags'] = preds3[:]

In [180]:
fist_dfmid = pd.DataFrame(df_test['image_name'][40669:61149])
fist_dfmid['tags'] = preds[40576:]

In [181]:
fist_df4 = pd.DataFrame(df_test['image_name'][61149:])
fist_df4['tags'] = preds4[:]

In [182]:
fist_df5 = fist_df.append(fist_df3).append(fist_dfmid).append(fist_df4)
fist_df5 = fist_df5.reset_index(drop=True)

In [183]:
fist_df5.head(5)

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,partly_cloudy primary
3,test_3,agriculture cultivation primary clear
4,test_4,partly_cloudy cloudy primary


In [184]:
fist_df5.to_csv('newsplanetsubmission2.csv', index=False)